In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
data=np.load('MDIVI/Mdivi1_7_18_2_3.npz')
vals,vecs=data['eigen_vals'],data['eigen_vecs']
index=np.argsort(vals)

In [ ]:
'''
Get the trailing elements of input x

@param x a 2nd order tensor with frames along the first axis
@param f the particular frame to pull from
@param i the parameter which determines how many trailing elements to pull
'''
def getTrail(x,f,i=10):
    return x[f,index[f,:i]]

In [ ]:
'''
Get the trailing elements of input x as a series iterating over frames

@param x a 2nd order tensor with frames along the first axis
@param f the parameter which determines how many frames to process
@param i the parameter which determines how many trailing elements to pull
'''
def getTrailSeries(x,f,i=10):
    out=np.zeros((f,i))
    for k in range(f):
        out[k]=getTrail(x,k,i)
    return out

In [ ]:
'''
Creates a lineplot leveragin getTrailSeries

@param x a 2nd order tensor with frames along the first axis
@param f the parameter which determines how many frames to process
@param i the parameter which determines how many trailing elements to pull
'''

def makeLine(x,f,i=10):
    fig=go.Figure()
    t=np.linspace(1,f,f)
    data=getTrailSeries(x,f,i)
    for k in range(1,i):
        fig.add_trace(go.Scatter(x=t, y=data[:,k]))
    return fig

In [ ]:
'''
Convert the data array into a fourier spectrum and display as a bar plot.
Note that we assume that the samples are collected over unit time, hence
the implicit sample frequency is #frames Hz. Note that in displaying the
spectra, if scaled then the constant term (freq 0) is omitted.

@param data a 2nd order tensor with frames along the first axis
@param i the parameter which determines how many trailing elements to pull.
       Note that the lowest trailing element, 0, is skipped.
@param cutoff the proportion (1/cutoff) of frequencies to display. That is
       to say, the output will display frequencies [0,#frames//(2*cutoff)]
@param scale if true, then the spectrum for each eigenvalue is scaled so
       that its value at freq 0 (constant) is 1. That is to say, the spectrum
       amplitudes become proportional to the amplitude of freq 0.
'''

def makeFourier(data,i=10,cutoff=5,scale=True):
    fig=go.Figure()
    N=len(data)
    data=data/data[:,1:2]
    
    #Freq space assuming eigenvalues sampled 
    #uniformally over unit time
    freq = np.linspace(0, 1, N)
    data=np.fft.rfft(data,axis=0)
    data=np.abs(data)/N
    if(scale):
        data=data/data[:1,:]

    for k in range(1,i):
        fig.add_trace(go.Bar(x=freq, y=data[1 if scale else 0:len(data)//cutoff,k]))
    return fig


In [ ]:
# Make a line plot of the trailing eigenvalues
makeLine(vals,len(vals),6).show()

# Fourier analysis of the trailing eigenvalues
data=getTrailSeries(vals,len(vals),7)
makeFourier(data,6,5).show()

# Look at the differences between eigenvalues4
# Otherwise known as eigengaps
# eigengap_k=eigenvalue_k-eigenvalue_k+1
data=np.diff(data)

# Fourier analysis of the differences
makeFourier(data,6,5,scale=False).show()